## SQL ##

*En esta fase del proceso de ETL  vamos a crear paso a paso una base de datos en sql desde python, para despúes crear las tablas con sus PK y sus FK.*

*Una vez todo esto este creado procedemos a poblarla desde python con nuestro df limpio y actualizado.*

Este es el codigo usado en sql para crear nuestra base de datos. *Este codigo se puede usar para crear la base de datos directamente desde SQL,pero vamos a crearla desde este entorno*

In [1]:
import pandas as pd
import mysql.connector
import numpy as np

Este es el codigo para crear desde python la base de datos en sql.

*Correr solo una vez*

In [2]:
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"#Aqui teneis que poner vuestra contraeña.
)

cursor = db.cursor()

# Crear la base de datos si no existe
cursor.execute('CREATE DATABASE IF NOT EXISTS AIRBNB')

# Conectar a la base de datos recién creada
db.database = 'AIRBNB'

# Crear la tabla Hosting
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Hosting (
        record_id BIGINT,
        titles TEXT,
        property_types TEXT,
        host_name TEXT,
        PRIMARY KEY (record_id)               
    );
''')

# Crear la tabla Description_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Description_ (
        id BIGINT,
        prices_per_night INT,
        check_in_hour TIME,
        check_out_hour TIME,
        total_hours_checkin INT,
        cleaning_fee INT,
        maximum_guests INT,
        camas INT,
        baños INT,
        dormitorios INT,
        FOREIGN KEY (id) REFERENCES Hosting(record_id)
    );
''')

# Crear la tabla Ratings
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Ratings (
        record_id BIGINT,
        ratings FLOAT,
        num_reviews FLOAT,
        sentiment TEXT,
        FOREIGN KEY (record_id) REFERENCES Hosting(record_id)
    );
''')

#Creamos la tabla de Services_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Services_ (
        service_id INT,
        service TEXT,
        PRIMARY KEY (service_id)
   );
''')
#Creamos la tabla de Service_ids
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Services_Hosting (
        service_id INT,
        record_id BIGINT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (record_id) REFERENCES Hosting (record_id)
);
''')   
#Creamos la tabla de Category
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Category (
        category_id INT,
        category TEXT,
        PRIMARY KEY (category_id)
 );
''')

#Creamos la tabla de Category_ids
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Category_Services (
        service_id INT,
        category_id INT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (category_id) REFERENCES Category(category_id)
);
''')   

# Cerrar el cursor y la conexión
cursor.close()
db.close()


Este es el codigo para la subida de nuestro df a la base de datos

In [3]:
#Hay que añadir el df de coments
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_cleaned_sentiment_sql.csv')
df_category = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_category.csv')
df_service = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_service.csv')
df_service_record = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_service_record.csv')
df_category_service = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_category_service.csv')

Dividimos las columnas del df en distintos df segun las tablas de nuestra base de datos.

*Elinamos las filas que tengan duplicada la columna record_id*

In [4]:
df_sin_duplicados = df.drop_duplicates(subset='record_id', keep=False)

In [5]:
df_sin_duplicados.head()

,urls,timestamp,record_id,titles,host_name,property_types,prices_per_night,location,ratings,num_reviews,cleaning_fee,dormitorios,camas,baños,maximum_guests,check_in_hour,check_out_hour,total_hours_checkin,sentiment_label_transformed
1,https://www.airbnb.es/rooms/127904085557632410...,2024-11-07 23:54:16,1279040855576324107,Este anfitrión tiene 979 evaluaciones de otros...,David,Habitación,46.0,"Barcelona, Catalunya, España",0.00,0.0,15.0,1.0,1.0,0.5,1.0,17:00,11:00,7.0,Positivo
3,https://www.airbnb.es/rooms/126566083301808951...,2024-11-07 23:54:16,1265660833018089515,Instalaciones y servicios nuevos a estrenar.es...,BLAU Student Housing,Alojamiento entero,100.0,"Barcelona, Catalunya, España",4.89,0.0,35.0,1.0,1.0,1.0,1.0,16:00,12:00,8.0,Positivo
8,https://www.airbnb.es/rooms/119763783122709724...,2024-11-07 23:54:16,1197637831227097242,El espacioel apartamento está así formado:⭑ser...,Raffaele,Alojamiento entero,115.0,"Barcelona, Catalunya, España",5.00,4.0,70.0,1.0,1.0,1.0,2.0,15:00,11:00,9.0,Positivo
18,https://www.airbnb.es/rooms/112315682764552247...,2024-11-07 23:54:16,1123156827645522473,Esta acogedora habitación cuenta con una cómod...,Manuel,Habitación,49.0,"Barcelona, Catalunya, España",4.65,23.0,0.0,1.0,1.0,0.5,1.0,15:00,12:00,9.0,Positivo
20,https://www.airbnb.es/rooms/31977858?adults=1&...,2024-11-07 23:54:16,31977858,hay una razón por la que hemos sido votados co...,St Christophers Barcelona,Habitación,18.0,"Barcelona, España",4.34,296.0,0.0,1.0,1.0,0.5,1.0,15:00,11:00,9.0,Positivo


In [6]:
df_Hosting = df_sin_duplicados[['record_id','titles','property_types','host_name']]

In [7]:
df_Hosting.head()

,record_id,titles,property_types,host_name
1,1279040855576324107,Este anfitrión tiene 979 evaluaciones de otros...,Habitación,David
3,1265660833018089515,Instalaciones y servicios nuevos a estrenar.es...,Alojamiento entero,BLAU Student Housing
8,1197637831227097242,El espacioel apartamento está así formado:⭑ser...,Alojamiento entero,Raffaele
18,1123156827645522473,Esta acogedora habitación cuenta con una cómod...,Habitación,Manuel
20,31977858,hay una razón por la que hemos sido votados co...,Habitación,St Christophers Barcelona


In [8]:
df_service_record.head()

,service,record_id
0,1,1263117591292794971
1,2,1263117591292794971
2,3,1263117591292794971
3,4,1263117591292794971
4,5,1263117591292794971


In [9]:
df_Description_ = df_sin_duplicados[['record_id', 'prices_per_night','check_in_hour','check_out_hour','cleaning_fee','total_hours_checkin','maximum_guests','camas','baños','dormitorios']]
df_Description_ = df_Description_.rename(columns={'record_id': 'id'})

In [10]:
df_Ratings = df_sin_duplicados[['record_id','ratings','num_reviews','sentiment_label_transformed']]
df_Ratings = df_Ratings.rename(columns={'sentiment_label_transformed': 'sentiment'})

Aqui nos aseguramos de que no haya ningun nan sustituyendolos todos por none (valor que si admite sql).

In [11]:
df_Description_ = df_Description_.replace({np.nan : None}).reset_index(drop = True) #Sustituimos todos los posibles nan por none para poder subirlos
df_Hosting = df_Hosting.replace({np.nan : None}).reset_index(drop = True)
df_Ratings = df_Ratings.replace({np.nan : None}).reset_index(drop = True)
df_service_record = df_service_record.replace({np.nan : None}).reset_index(drop = True)
df_category_service = df_category_service.replace({np.nan : None}).reset_index(drop = True)

Aqui procedemos a la subida de las distintas tablas.

Hosting

In [12]:
column_names= ['record_id','titles','property_types','host_name']

In [13]:
database = "AIRBNB"
table_name = "Hosting"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Hosting.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Hosting[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 848 filas


Description_

In [14]:
column_names = ['id', 'prices_per_night','check_in_hour','check_out_hour','cleaning_fee','total_hours_checkin','maximum_guests','camas','baños','dormitorios']

In [15]:
database = "AIRBNB"
table_name = "Description_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Description_.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Description_[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 848 filas


Ratings

In [16]:
#Hay que añadir los coments
column_names= ['record_id','ratings','num_reviews', 'sentiment']

In [17]:
df_Ratings["record_id"] = df_Ratings["record_id"].apply(lambda x : str(x))

In [18]:
database = "AIRBNB"
table_name = "Ratings"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names
print(column_names)

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Ratings.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"
print(insert_query)

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Ratings[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

('record_id', 'ratings', 'num_reviews', 'sentiment')
INSERT INTO Ratings (record_id, ratings, num_reviews, sentiment) VALUES (%s, %s, %s, %s)
Añadidas: 848 filas


Services_

In [19]:
column_names = ['service_id', 'service']

In [20]:
database = "AIRBNB"
table_name = "Services_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_service.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_service[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 1143 filas


Services_Hosting

In [21]:
column_names = ['service_id', 'record_id']

In [22]:
df_service_record = df_service_record.rename(columns={'service': 'service_id'})

In [23]:
df_service_record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75056 entries, 0 to 75055
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   service_id  75056 non-null  int64
 1   record_id   75056 non-null  int64
dtypes: int64(2)
memory usage: 1.1 MB


In [24]:
df_service_record["service_id"] = df_service_record["service_id"].apply(lambda x : str(x))

In [25]:
df_service_record["record_id"] = df_service_record["record_id"].apply(lambda x : str(x))

In [26]:
database = "AIRBNB"
table_name = "Services_Hosting"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraseña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_service_record.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_service_record[list(column_names)].values] 

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values) #Parece que es aqui donde da el error,no funciona ni en int ni en obj

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`airbnb`.`services_hosting`, CONSTRAINT `services_hosting_ibfk_2` FOREIGN KEY (`record_id`) REFERENCES `hosting` (`record_id`))

Category

In [ ]:
column_names = ['category_id', 'category']

In [ ]:
database = "AIRBNB"
table_name = "Category"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_category.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_category[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 13 filas


Category_Services

In [ ]:
column_names = ['category_id', 'service_id']

In [ ]:
df_category_service = df_category_service.rename(columns={'category': 'category_id'})
df_category_service = df_category_service.rename(columns={'service': 'service_id'})

In [ ]:
df_category_service.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   category_id  1150 non-null   int64
 1   service_id   1150 non-null   int64
dtypes: int64(2)
memory usage: 18.1 KB


In [ ]:
df_category_service["service_id"] = df_category_service["service_id"].apply(lambda x : str(x))

In [ ]:
df_category_service["category_id"] = df_category_service["category_id"].apply(lambda x : str(x))

In [ ]:
database = "AIRBNB"
table_name = "Category_Services"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_category_service.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_category_service[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 1150 filas


# Actualizar base de datos

**Inserta solo los datos que no existan en la base de datos.**

*ruta_csv*: Ruta del archivo CSV que contiene los nuevos datos.

*conexion_db*: Objeto de conexión a la base de datos MySQL (usando mysql.connector).

*tabla*: Nombre de la tabla en la que se insertarán los datos.

*columnas_unicas*: Lista de columnas que definen la unicidad de los registros. 

In [ ]:
def actualizar_base_datos(ruta_csv, conexion_db, tabla, columnas_unicas):
    
    # Crear cursor
    cursor = conexion_db.cursor(dictionary=True)

    # Cargar datos desde el archivo CSV
    nuevos_datos = pd.read_csv(ruta_csv)

    # Iterar sobre cada fila de datos nuevos
    for _, fila in nuevos_datos.iterrows():
        # Construir la condición para la consulta de verificación
        condicion = " AND ".join([f"{col} = %s" for col in columnas_unicas])
        valores_condicion = [fila[col] for col in columnas_unicas]
        query = f"SELECT COUNT(*) as conteo FROM {tabla} WHERE {condicion}"
        
        # Ejecutar la consulta de verificación
        cursor.execute(query, valores_condicion)
        resultado = cursor.fetchone()
        
        # Si no hay registros duplicados, insertar la fila
        if resultado['conteo'] == 0:
            columnas = ", ".join(fila.index)
            valores = ", ".join(["%s"] * len(fila))
            insert_query = f"INSERT INTO {tabla} ({columnas}) VALUES ({valores})"
            cursor.execute(insert_query, list(fila))
            conexion_db.commit()
            print(f"Datos insertados: {fila[columnas_unicas].to_dict()}")
        else:
            print(f"Duplicado encontrado, datos no insertados: {fila[columnas_unicas].to_dict()}")

    # Cerrar el cursor al finalizar
    cursor.close()

Ejemplo

In [ ]:
"""
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"
 )
db.database = 'AIRBNB'
actualizar_base_datos('ruta.csv', db, 'nombre_tabla', ['columnas'])
"""

'\ndb = mysql.connector.connect(\n     host="localhost",\n     user="root",\n     password="contraseña123"\n )\ndb.database = \'AIRBNB\'\nactualizar_base_datos(\'ruta.csv\', db, \'nombre_tabla\', [\'columnas\'])\n'

# Extracción de datos

In [ ]:
def extraer_datos_mysql(host, user, password, database, query):
    # Conectarse a la base de datos
    db = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Crear cursor para ejecutar la consulta
    cursor = db.cursor()

    # Ejecutar la consulta
    cursor.execute(query)
    
    # Obtener los nombres de las columnas
    column_names = cursor.column_names
    print("Nombres de las columnas:", column_names)

    # Guardar y mostrar los datos de la tabla
    data = cursor.fetchall()
    for fila in data:
        print(fila)

    # Cerrar el cursor y la conexión
    cursor.close()
    db.close()


Hosting

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Hosting;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('record_id', 'titles', 'property_types', 'host_name')
(72436, 'Disponemos de 19 apartamentos de 45 a 57m2 situados entre la planta baja y la 3ª. tienen las mismas características: 1 habitación, 1 baño, salón-cocina, con decoraciones diferentes. no todos recogidos en la galería de fotos. apartamentos estilosos delante de la playa.acceso a la piscina con medidas 4x5.75 y 3x2000171/000172/000182/011441/011442/011440/011443/011444/010962/002917/010961/005109/005102/010969/005112/005100/010968/01097/010999.el espaciolos apartamentos del lugaris beach presentan un diseño contemporáneo y cuentan con ventanales, conexión a internet de alta velocidad gratuita y cocina privada equipada con horno, cafetera nespresso y nevera. además, constan de sala de estar, tv de pantalla plana con canales internacionales, lavadora y secadora de ropa. también incluyen 1 paquete de bienvenida con productos básicos como café, té, leche, azúcar, sal y aceite. además de un botella de agua 

Description_

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Description_;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('id', 'prices_per_night', 'check_in_hour', 'check_out_hour', 'total_hours_checkin', 'cleaning_fee', 'maximum_guests', 'camas', 'baños', 'dormitorios')
(1279040855576324107, 46, datetime.timedelta(seconds=61200), datetime.timedelta(seconds=39600), 7, 15, 1, 1, 1, 1)
(1265660833018089515, 100, datetime.timedelta(seconds=57600), datetime.timedelta(seconds=43200), 8, 35, 1, 1, 1, 1)
(1197637831227097242, 115, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=39600), 9, 70, 2, 1, 1, 1)
(1123156827645522473, 49, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=43200), 9, 0, 1, 1, 1, 1)
(31977858, 18, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=39600), 9, 0, 1, 1, 1, 1)
(1268163818980729724, 147, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=43200), 9, 0, 4, 2, 1, 2)
(1086018140861723540, 122, datetime.timedelta(seconds=57600), datetime.timedelta(seconds=43200), 8, 40, 3, 2, 1, 1)
(1278815157116605567, 142, d

Ratings

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Ratings;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('record_id', 'ratings', 'num_reviews', 'sentiment')
(1279040855576324107, 0.0, 0.0, 'Positivo')
(1265660833018089515, 4.89, 0.0, 'Positivo')
(1197637831227097242, 5.0, 4.0, 'Positivo')
(1123156827645522473, 4.65, 23.0, 'Positivo')
(31977858, 4.34, 296.0, 'Positivo')
(1268163818980729724, 0.0, 0.0, 'Positivo')
(1086018140861723540, 5.0, 3.0, 'Positivo')
(1278815157116605567, 0.0, 0.0, 'Positivo')
(40577359, 4.47, 103.0, 'Positivo')
(22548858, 4.84, 0.0, 'Positivo')
(1277720203892360783, 0.0, 0.0, 'Positivo')
(1248794356730472452, 0.0, 0.0, 'Positivo')
(39788335, 4.81, 48.0, 'Positivo')
(1100921852236177943, 4.71, 34.0, 'Positivo')
(12634537, 4.75, 0.0, 'Positivo')
(1210293681811098298, 0.0, 0.0, 'Positivo')
(850684095339937556, 5.0, 5.0, 'Positivo')
(10038461, 4.87, 0.0, 'Positivo')
(1273185111265704381, 0.0, 0.0, 'Positivo')
(961442045210294120, 4.51, 81.0, 'Positivo')
(1109989, 4.8, 573.0, 'Positivo')
(549976913937241380, 4.46, 26.0, 'Positivo')
(80004, 4.66,

Services

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Services_;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('service_id', 'service')
(1, 'secador de pelo')
(2, 'productos de limpieza')
(3, 'champú')
(4, 'gel de ducha')
(5, 'agua caliente')
(6, 'lavadora')
(7, 'perchas')
(8, 'ropa de cama')
(9, 'persianas o cortinas opacas')
(10, 'plancha')
(11, 'tendedero para ropa')
(12, 'espacio para guardar la ropa: armario')
(13, 'tv')
(14, 'aire acondicionado')
(15, 'calefacción central')
(16, 'botiquín')
(17, 'wifi')
(18, 'cocinacocina disponible para el uso de los huéspedes')
(19, 'frigorífico')
(20, 'microondas')
(21, 'congelador')
(22, 'horno')
(23, 'hervidor de agua')
(24, 'entrada independienteentrada por otra calle o edificio')
(25, 'lavandería cercana')
(26, 'patio o balcón')
(27, 'comedor al aire libre')
(28, 'ascensorel alojamiento o edificio dispone de un ascensor de al menos 132\xa0centímetros de profundidad cuya puerta mide como mínimo 81\xa0centímetros de ancho.')
(29, 'admite mascotaslos animales de asistencia siempre están permitidos')
(30, 'se permite dejar el 

Category

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Category;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('category_id', 'category')
(1, 'baño')
(2, 'dormitorio y lavandería')
(3, 'entretenimiento')
(4, 'calefacción y refrigeración')
(5, 'seguridad en el hogar')
(6, 'internet y oficina')
(7, 'cocina y comedor')
(8, 'características de la ubicación')
(9, 'exterior')
(10, 'aparcamiento e instalaciones')
(11, 'servicios')
(12, 'privacidad y seguridad')
(13, 'para familias')
